# STAGE 1 - Dataset creation

## Phase 1: Create the data frame from a file downloaded from Scopus or Web of Science

### Input
 - **filename**: name of the file downloaded from Web of Sciece with .txt extension. The file must be in the same folder of this Notebook.

### Output
 - "Dataset_input.xslx" file: a DataFrame in which each row represents a paper and the columns are:
     - text id; Authors; Article title; Abstract; Source title; Pubblication year; Journal abbreviation; Journal ISO abbreviation; References; Total number of citations; Number of internal citations; References internal id; DOI.

In [ ]:
import preprocessing_functions as prp
prp.read_wos_txt(filename='your_file_path.txt')

## Or

### Input
 - **filename**: name of the file downloaded from Scopus with .csv extension. The file must be in the same folder of this Notebook.

### Output
 - "Dataset_input.xslx" file: a DataFrame in which each row represents a paper and the columns are:
     - text id; Authors; Article title; Abstract; Source title; Pubblication year; Journal abbreviation; References; Total number of citations; Number of internal citations; References internal id; DOI.

In [ ]:
import preprocessing_functions as prp

read_scopus_csv(filename='your_file_path.csv')

# Phase 2: Add the top journal flags
### Inputs:
 - **filename**: file to indicate which are the Top journals. The default file is 'TopJournal_list.txt' and the list was taken from https://journalranking.org considering all the journals ranked as 3, 4, and 4* according to the ABS ranking. If you want to use a different file, create a file .txt with the list of journals, where each journal name is divided by newline as in the file 'TopJournal_list.txt'.


 - **df_file**: name of the file with .xlsx extension. The file is a data frame created in the previuos step, named 'Dataset_input.xlsx'. The file is a data frame with a column, named 'source_title', that refers to the name of the journal where the paper was pubblished.

### Output:
- 'Dataset_input.xslx' file: the input data frame  with a new column ('TOPJ') to indicate if the journal is top or not.

In [ ]:
slr.add_top_journal(filename="TopJournal_list.txt", df_file='Dataset_input.xlsx')

# Phase 3: Clean the texts
### Input:
- **filename**: name of the file with .xlsx extension. The file is a data frame as created in the previuos step, named 'Dataset_input.xlsx'.
    - Note: each text (abstract) should be a single string.

### Output:
- 'Dataset_input.xslx' file: the input data frame with a new column ('clean_text) with the cleaned version of the abstract.

In [ ]:
slr.preprocess(filename='Dataset_input.xlsx')

# STAGE 2 - Analysis

# Phase 3: Topic definition and document—topic associations
##  Input values required to run the analysis:

- **file_name**: name of the file created in Stage 1 ("Dataset_Input.xlsx").
    - If you want to create your data frame skipping Stage 1, note that the input data frame must have the following columns:
        - 'text_id': the identifier of the document, e.g. "d1".
        - 'Total number of citations': total number of citations.
        - 'Number of internal citations': number of citations within the dataset.
        - 'References internal id': string of 'text_id' separated by space reporting the documents that appear as references, e.g. "d1 d2 d3 d4 d5".
        - 'TopJ': boolean ("Y" or "N") if the pubblication belongs to Top Journal or not.
        - 'clean_text': pre—processed text as string; words are separated by space.
    


- **method_w**: correction method for multiple testing problem: 'bonf' (Bonferroni correction) or 'fdr' (False Discovery Rate, Benjamini Y and Hochberg Y, 1995); the default value is 'fdr'.


- **threshold_w**: value of the threshold to construct the Statistically Validated Network of words; the defaut value is 0.01.


- **threshold_d**: value of the threshold for document-topic associations with Bonferroni correction method, the defaut value is 0.01.


- **threshold_0**: value of the threshold to calculate the associations between documents and the topic named 'General' with the Bonferroni correction method; the defaut value is 0.01.


## Outputs:
- **SVN words.txt**: file with the edges list of the Statistically Validated Network of words; each row represents a link between two nodes ('source' and 'target') with its p-value and Pearson correlation coefficient. The file consists of four columns separated by '\t':
    - source, target, p-values, weight (Pearson correlation coefficient).
    
    
- **Topic definition.xlsx**: data frame describing the Topics found as community of words in the Statistically Validated Network. The data frame has the following columns:
    - 'topic': the identifier of the topic, e.g. "topic_1".
    - 'word': the word in stemming form.
    - 'modularity contribution': the importance of the node (word) within the community (topic) in terms of modularity contribution.
    - 'original words': the list of words before the stemming procedure.


- **Topic Document association.xlsx**: data frame describing the associations between documents and topics; "topic_0" represents the 'General' topic.  The data frame has the following columns:
    - 'text_id': the identifier of the document, e.g. "d1".
    - 'topic': the identifier of the topic, e.g. "topic_1".
    - 'p-value': the p-value of the over—expression of the topic within the document.
    - 'correlation': Pearson correlation coefficient of the over—expression of the topic within the document.

In [ ]:
import toolkit_functions as tlk

In [ ]:
tlk.run_analysis(file_name='Dataset_Input.xlsx', method_w='fdr', threshold_w=0.01,
                 threshold_d=0.01, threshold_0=0.01)

# Phase 4: Statistics of the selected topics

## Input values required:

- **file_name1**: file with .xlsx extension. The file 'Dataset_Input.xlsx' was used as input in Phase 1.


- **file_name2**:  file with .xlsx extension. The file 'Topic Document association.xlsx' was obtained in Phase 1.


- **file_name3**: file with .xlsx extension. The file 'Topic definition.xlsx' was obtained in Phase 1.


- **file_name4**: file with .xlsx extension. The file 'topic_label_example.xlsx' was created to store the labels of topics in STAGE 2 - Step 2. The file must be a data frame with two columns called 'topic' and 'label'. The column 'topic' contains the id of topics as in the file 'Topic definition.xlsx'. The column 'label' contains the name you assigned to the topic selected. The file 'topic_label_example.xlsx' is an example.

## Outputs:

- **stats_topic.xlsx**: data frame describing the topics and some related statistics.

In [ ]:
tlk.run_stats(file_name1='Dataset_Input.xlsx', file_name2='Topic Document association.xlsx', 
              file_name3='Topic definition.xlsx', file_name4='topic_label_example.xlsx')



# STAGE 3 - Plots

# Phase 5: Plot the selected topics

## Input:
- **df**: file with .xlsx extension. The output file 'stats_topic.xlsx' of Phase 4.
- **name**: name of the file to save the plot.

## Output:
- Scatter-plot of topics along two dimensions: 'ratio of citations' (x-axis) and 'ratio of top journals' (y-axis). 'ratio of citations' is obtained as the 'Average number of citations within the dataset' divided by 'Average number of citations'. 'ratio of top journals' is obtained as the 'Number of papers from top journals over—expressed' divided by 'Number of papers over—expressed'.

In [ ]:
tlk.plot_stats_1(df='stats_topic.xlsx', name='topic_overview_1.pdf')

## Input:
- **df**: file with .xlsx extension. The output file 'stats_topic.xlsx' of Phase 4.
- **name**: name of the file to save the plot.

## Output:
- Scatter-plot of topics along two dimensions: 'Average number of citations within the dataset' (x-axis) and 'Average number of citations' (y-axis). 

In [ ]:
tlk.plot_stats_2(df='stats_topic.xlsx', name='topic_overview_2.pdf')